In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import datetime
from urllib.request import urlopen, Request

In [2]:
r = Request('https://th.investing.com/economic-calendar/', headers={'User-Agent': 'Mozilla/5.0'})
time.sleep(2)
response = urlopen(r).read()
soup = BeautifulSoup(response, "html.parser")
table = soup.find_all(class_ = "js-event-item")
result = []

In [3]:
def event_calendar():
    
    for bl in table:
        event_datetime = bl.get('data-event-datetime', '').strip()
        currency = bl.find(class_="left flagCur noWrap").text.strip()
        intensity_divs = bl.find_all(class_="left textNum sentiment noWrap")
        event = bl.find(class_="left event").text.strip()
        intencity_val = 0
        true_count = 0

        for intence in intensity_divs:
            _true = intence.find_all(class_="grayFullBullishIcon")
            _false = intence.find_all(class_="grayEmptyBullishIcon")

            true_count = len(_true)

            if true_count == 3:
                intencity_val = 3
            elif true_count == 2:
                intencity_val = 2
            else :
                intencity_val = 1
                
        event_datetime = event_datetime.split(' ')
        date=event_datetime[0]
        time=event_datetime[1]
        
        
        result.append({'currency' : currency, 'date' : date, 'time' : time, 'intensity' : intencity_val, 'event':event})

    return result

In [4]:
news = event_calendar()
news_df = pd.DataFrame(news)

In [5]:
display(news_df.shape)
display(news_df.head(10))

(53, 5)

,currency,date,time,intensity,event
0,CAD,2025/06/19,00:30:00,1,BOC Summary of Deliberations
1,EUR,2025/06/19,00:30:00,2,คำกล่าวของนายเบิร์คฮาร์ด บัลซ์ (Balz) แห่งธนาค...
2,BRL,2025/06/19,00:30:00,1,ดัชนีกระแสการแลกเปลี่ยนเงินตราต่างประเทศของบราซิล
3,USD,2025/06/19,01:00:00,2,การคาดการณ์อัตราดอกเบี้ย - ปีที่ 1 (ไตรมาส 2)
4,USD,2025/06/19,01:00:00,2,การคาดการณ์อัตราดอกเบี้ย - ปีที่ 2 (ไตรมาส 2)
5,USD,2025/06/19,01:00:00,2,การคาดการณ์อัตราดอกเบี้ย - ปัจจุบัน (ไตรมาส 2)
6,USD,2025/06/19,01:00:00,2,การคาดการณ์อัตราดอกเบี้ย - ระยะยาว (ไตรมาส 2)
7,USD,2025/06/19,01:00:00,3,การคาดการณ์เศรษฐกิจของคณะกรรมการนโยบายธนาคารกล...
8,USD,2025/06/19,01:00:00,3,แถลงการณ์ของคณะกรรมการนโยบายการเงินธนาคารกลางส...
9,USD,2025/06/19,01:00:00,3,การตัดสินใจเกี่ยวกับอัตราดอกเบี้ย


In [6]:
now = datetime.datetime.now()
date_time = now.strftime("%Y-%m-%d %H-%M-%S").strip().replace(' ', '_')
news_df.to_csv(f'Data/Event_Calendar/{date_time}.csv', index=False)

In [7]:
print(news_df[news_df['intensity'] == 3][['currency', 'event']])

   currency                                              event
7       USD  การคาดการณ์เศรษฐกิจของคณะกรรมการนโยบายธนาคารกล...
8       USD  แถลงการณ์ของคณะกรรมการนโยบายการเงินธนาคารกลางส...
9       USD                  การตัดสินใจเกี่ยวกับอัตราดอกเบี้ย
11      USD  การแถลงข่าวต่อสื่อมวลชนของคณะกรรมการกำหนดนโยบา...
30      CHF      การตัดสินใจเกี่ยวกับอัตราดอกเบี้ย  (ไตรมาส 2)
33      CHF  การแถลงข่าวต่อสื่อมวลชนของธนาคารกกลางสวิตเซอร์...
50      GBP         การตัดสินใจเกี่ยวกับอัตราดอกเบี้ย  (มิ.ย.)


In [9]:
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

load_dotenv("../SentimentAnalysis/GPT/secret.env")

mongo_connection_string = os.getenv("MONGO_CONNECTION_STRING")

try:
    client = MongoClient(mongo_connection_string)
    db = client['stock_news_db']
    collection = db['event_calendar']

    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")

except Exception as e:
    print(f"An error occurred: {e}")

Pinged your deployment. You successfully connected to MongoDB!


In [10]:
complete_dict=news_df.to_dict(orient='records')

result = collection.insert_many(complete_dict,ordered=True)
print(f"Successfully inserted document with id: {result.inserted_ids}")

Successfully inserted document with id: [ObjectId('68537768c3123f3610cb024d'), ObjectId('68537768c3123f3610cb024e'), ObjectId('68537768c3123f3610cb024f'), ObjectId('68537768c3123f3610cb0250'), ObjectId('68537768c3123f3610cb0251'), ObjectId('68537768c3123f3610cb0252'), ObjectId('68537768c3123f3610cb0253'), ObjectId('68537768c3123f3610cb0254'), ObjectId('68537768c3123f3610cb0255'), ObjectId('68537768c3123f3610cb0256'), ObjectId('68537768c3123f3610cb0257'), ObjectId('68537768c3123f3610cb0258'), ObjectId('68537768c3123f3610cb0259'), ObjectId('68537768c3123f3610cb025a'), ObjectId('68537768c3123f3610cb025b'), ObjectId('68537768c3123f3610cb025c'), ObjectId('68537768c3123f3610cb025d'), ObjectId('68537768c3123f3610cb025e'), ObjectId('68537768c3123f3610cb025f'), ObjectId('68537768c3123f3610cb0260'), ObjectId('68537768c3123f3610cb0261'), ObjectId('68537768c3123f3610cb0262'), ObjectId('68537768c3123f3610cb0263'), ObjectId('68537768c3123f3610cb0264'), ObjectId('68537768c3123f3610cb0265'), ObjectId(